# Driver Identification based on Sánchez et al. 2020

Sánchez, S.H., Pozo, R.F. and Gómez, L.A.H., 2020. Driver Identification and Verification From Smartphone Accelerometers Using Deep Neural Networks. IEEE Transactions on Intelligent Transportation Systems.

https://ieeexplore.ieee.org/abstract/document/9145829

# Prepare

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os
import seaborn as sns
from scipy import stats
from sklearn.neural_network import MLPClassifier
from sklearn.metrics.classification import accuracy_score, recall_score, f1_score, precision_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn import preprocessing
import keras
from keras.layers import Dense, Input, Reshape, Flatten, BatchNormalization, LeakyReLU, GRU
from keras.layers.convolutional import Conv1D
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback
from keras.layers import GlobalAveragePooling2D
from keras.models import Model,Sequential
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.utils import to_categorical
from keras.optimizers import Adam

import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
# access to my google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#################### DOWNLOAD AND UNZIP FILE SAVED IN DRIVE ####################

!pip install -U -q PyDrive


# HERE YOUR FILE ID ( GET IT WITH THE SHARING URL: https://drive.google.com/open?id=1MWZpk6SRmqBUGjwcdaEzoVDm7mUKOnci )

zip_id = '1MWZpk6SRmqBUGjwcdaEzoVDm7mUKOnci'



from pydrive.auth import GoogleAuth

from pydrive.drive import GoogleDrive

from google.colab import auth

from oauth2client.client import GoogleCredentials

import zipfile, os


# 1. Authenticate and create the PyDrive client.

auth.authenticate_user()

gauth = GoogleAuth()

gauth.credentials = GoogleCredentials.get_application_default()

drive = GoogleDrive(gauth)



# DOWNLOAD ZIP

print ("Downloading zip file")

myzip = drive.CreateFile({'id': zip_id})

myzip.GetContentFile('DrEftekhari.zip')



# UNZIP ZIP

print ("Uncompressing zip file")

zip_ref = zipfile.ZipFile('DrEftekhari.zip', 'r')

zip_ref.extractall()

zip_ref.close()

Uncompressing zip file


# Dataset

In [ ]:
import os
from datetime import datetime, timedelta
import re
import pandas as pd
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sn
import math
from sklearn import preprocessing
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.metrics import classification_report


class Utils:

    def __init__(self, sample_rate, train_segments, test_segments):
        self.all_features = ['x-accelerometer', 'y-accelerometer', 'z-accelerometer',
                             'GRAVITY X (m/s²)', 'GRAVITY Y (m/s²)', 'GRAVITY Z (m/s²)',
                             'LINEAR ACCELERATION X (m/s²)', 'LINEAR ACCELERATION Y (m/s²)',
                             'LINEAR ACCELERATION Z (m/s²)',
                             'x-gyroscope', 'y-gyroscope', 'z-gyroscope',
                             'LIGHT (lux)',
                             'MAGNETIC FIELD X (μT)', 'MAGNETIC FIELD Y (μT)', 'MAGNETIC FIELD Z (μT)',
                             'ORIENTATION Z (azimuth °)', 'ORIENTATION X (pitch °)', 'ORIENTATION Y (roll °)',
                             'LOCATION Latitude : ',
                             'LOCATION Longitude : ',
                             'LOCATION Altitude ( m)',
                             'LOCATION Altitude-google ( m)',
                             'LOCATION Speed ( Kmh)',
                             'LOCATION Accuracy ( m)',
                             'LOCATION ORIENTATION (°)',
                             'Satellites in range',
                             'Time since start in ms',
                             'timestamp']
        self.show_toolbar = True
        self.sample_rate = sample_rate
        self.train_segments = train_segments
        self.test_segments = test_segments

    def read_data(self, db_path_prefix, file_name, features, driver_i):
        db_path = db_path_prefix + file_name
        db_new_path = db_path_prefix + "cleaned_by_acc/"

        if os.path.exists(db_new_path + file_name):
            data = pd.read_csv(db_new_path + file_name, low_memory=False)
            original_size = pd.read_csv(db_path, low_memory=False).shape[0]
            driving_size = data.shape[0]
            stay_size = original_size - driving_size
        else:
            data = pd.read_csv(db_path, low_memory=False)
            data.columns = self.all_features

            original_size = data.shape[0]

            def aggregate(point1, point2):
                return math.sqrt(math.pow(point1[0] - point2[0], 2) +
                                 math.pow(point1[1] - point2[1], 2) +
                                 math.pow(point1[2] - point2[2], 2))

            staypoints = [0]
            points_acc = list(zip(data['x-accelerometer'], data['y-accelerometer'], data['z-accelerometer']))
            for j in range(0, len(points_acc) - 12):
                node = points_acc[j]
                add = True
                for j2 in range(j + 1, j + 12):
                    if aggregate(node, points_acc[j2]) > 0.5:
                        add = False
                if add:
                    staypoints.append(j)

            if not staypoints.__contains__(len(points_acc) - 1):
                staypoints.append(len(points_acc) - 1)

            stay_size = len(staypoints)

            data = data.drop(index=data['x-accelerometer'][staypoints].index)

            driving_size = data.shape[0]

            if not os.path.exists(db_new_path):
                os.makedirs(db_new_path)
            data.to_csv(db_new_path + file_name, index=False)
        
        if features.__contains__('gyroscope'):
          data['gyroscope'] = np.linalg.norm(data[['x-gyroscope', 'y-gyroscope', 'z-gyroscope']].values, axis=1)
        
        missing_features = self.all_features.copy()
        for feature in features:
          if feature != 'gyroscope':
            missing_features.remove(feature)
        data = data.drop(columns=missing_features)

        data = data.fillna(data.mean())
        # data = data.dropna()

        clean_driving_size = data.shape[0]

        template = "{0:20}{1:20}{2:20}{3:20}{4:20}"
        if self.show_toolbar:
            self.show_toolbar = False
            print(template.format("driver_id: ", "original_size: ", "stay_size: ", "driving_size: ",
                                  "cleaned_driving_size: "))
        print(template.format(str(driver_i),
                              str(timedelta(seconds=int(original_size / self.sample_rate))),
                              str(timedelta(seconds=int(stay_size / self.sample_rate))),
                              str(timedelta(seconds=int(driving_size / self.sample_rate))),
                              str(timedelta(seconds=int(clean_driving_size / self.sample_rate)))))

        return self.split_to_train_test(data)

    def split_to_train_test(self, data):
        flag = True
        for (start, end) in self.train_segments:
            if flag == True:
              flag = False
              train_data = data[start:end]
            else:
              train_data = pd.concat([data[start:end], train_data])
        
        flag = True
        for (start, end) in self.test_segments:
            if flag == True:
              flag = False
              test_data = data[start:end]
            else:
              test_data = pd.concat([data[start:end], test_data])
        
        return train_data, test_data

    def save_result(self, saving_path, result, data, running_time):
        if not os.path.exists(saving_path):
            os.makedirs(saving_path)
        
        # Save to file
        with open(saving_path + 'statistics.txt', 'a') as f:
            f.write('\n==========***==========\n' +
                    datetime.now().strftime("%Y:%m:%d %H:%M:%S") +
                    '\n' +
                    'running time :' + str(running_time.seconds) + " seconds" +
                    '\n')
            f.write(str(data))
            f.write('\n')
            f.write(str(result))
            f.write('\n')

# Segmentation

In [ ]:
import numpy as np


class Segmenter:

    def __init__(self, window_size=90, window_overlap_size=45, sampling=2):
        self.window_size = window_size
        self.overlapping = window_overlap_size
        self.sampling = sampling

    def transfer(self, dataset, features):
        print("segmenting data with " + str(len(dataset)) + " points")
        segments, labels = self.segment_signal(dataset, features)
        new_dataset = []
        print("making " + str(len(segments)) + " segments")
        for segment in segments:
            row = []
            for feature_i in range(len(features)):
                row.append(segment[feature_i])
            new_dataset.append(row)

        new_dataset = np.array(new_dataset)
        return new_dataset, labels

    def windows(self, data):
        start = 0
        while start < data.count():
            yield int(start), int(start + self.window_size)
            start += (self.window_size - self.overlapping)

    def segment_signal(self, dataset, features):
        segments = []
        labels = []
        for class_i in np.unique(dataset["id"]):
            subset = dataset[dataset["id"] == class_i]
            for (start, end) in self.windows(subset["id"]):
                feature_slices = []
                for feature in features:
                    feature_slices.append(subset[feature][start:end].tolist())
                if len(feature_slices[0]) == self.window_size:
                    segments.append(feature_slices)
                    labels.append(class_i)
        return np.array(segments), np.array(labels)


# VARIABLES

In [ ]:
def initialization(n_driver, train_segments, test_segments):
  global features
  features = ['x-accelerometer', 'y-accelerometer', 'gyroscope']
  # 'x-accelerometer','y-accelerometer','z-accelerometer','x-gyroscope','y-gyroscope','z-gyroscope', 'gyroscope'
  global db_path_prefix
  db_path_prefix = ''
  global sample_rate
  sample_rate = 2

  global window_size
  window_size = 224
  global overlapping
  overlapping = int(window_size * 0.25)

  global utils
  utils = Utils(sample_rate=sample_rate, train_segments=train_segments, test_segments=test_segments)

#SEGMENTATION

In [ ]:
def read_data():
  global train_dataset
  train_dataset = pd.DataFrame()
  global test_dataset
  test_dataset = pd.DataFrame()
  for i in range(211 - n_driver, 211):
    train_temp_dataset, test_temp_dataset = utils.read_data(db_path_prefix, str(i) + '.1.csv', features, i)
    train_temp_dataset['id'] = i
    test_temp_dataset['id'] = i
    train_dataset = pd.concat([train_dataset, train_temp_dataset])
    test_dataset = pd.concat([test_dataset, test_temp_dataset])

In [ ]:
def replace_ids():
  replace_y_numbers = {k: v for v, k in enumerate(sorted(set(train_dataset.iloc[:, -1])))}
  train_dataset.iloc[:, -1] = train_dataset.iloc[:, -1].replace(replace_y_numbers)
  test_dataset.iloc[:, -1] = test_dataset.iloc[:, -1].replace(replace_y_numbers)

In [ ]:
def Standardization(X_train, X_test):
  CX_train = X_train.copy()
  CX_test = X_test.copy()

  scaler = preprocessing.StandardScaler()
  scaler = scaler.fit(CX_train)
  NCX_train = scaler.transform(CX_train)
  NCX_test = scaler.transform(CX_test)

  return NCX_train, NCX_test

In [ ]:
def data_standardization():
  global n_train_dataset
  global n_test_dataset
  n_train_dataset, n_test_dataset = Standardization(train_dataset.iloc[:, :-1], test_dataset.iloc[:, :-1])
  n_train_dataset = pd.DataFrame(n_train_dataset, columns=features)
  n_test_dataset = pd.DataFrame(n_test_dataset, columns=features)
  n_train_dataset['id'] = train_dataset.iloc[:, -1].tolist()
  n_test_dataset['id'] = test_dataset.iloc[:, -1].tolist()

In [ ]:
def feature_extraction():
  featureExtractor = Segmenter(window_size=window_size, window_overlap_size=overlapping)

  global SX_train
  global y_train
  SX_train, y_train = featureExtractor.transfer(n_train_dataset, features)
  global SX_test
  global y_test
  SX_test, y_test = featureExtractor.transfer(n_test_dataset, features)

#Model 1D to 2D

In [ ]:
class OnetoTwoDimensionNetwork():
    def __init__(self, rows, cols, channels, classes):
        # Input shape
        self.img_rows = rows
        self.img_cols = cols
        self.channels = channels
        self.classes = classes
        self.img_shape = (self.img_rows, self.img_cols)

        optimizer = Adam(0.001, 0.5)

        self.discriminator = self.build_discriminator()
        self.generator = self.build_generator()

        # The generator takes signal as input and generates imgs
        z = Input(shape=self.img_shape)
        img = self.generator(z)

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        
    def build_generator(self):
      
        model = Sequential()
        
        model.add(Conv1D(64, kernel_size=3, strides=1, activation='relu', padding="same", kernel_regularizer='l2'))
        model.add(BatchNormalization(momentum=0.9))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv1D(224, kernel_size=3, strides=1, activation='relu', padding="same", kernel_regularizer='l2'))
        model.add(BatchNormalization(momentum=0.9))
        model.add(LeakyReLU(alpha=0.2))

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def build_discriminator(self):
        img_shape = (224, 224)
        
        model = Sequential()

        model.add(Flatten(input_shape=img_shape))
        model.add(Dense(self.classes, activation='softmax'))

        img = Input(shape=img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, X_train, y_train, X_valid, y_valid, batch_size=128):
        logdir = "logs/scalars/" + datetime.now().strftime("%Y.%m.%d-%H:%M:%S")
        tb = keras.callbacks.TensorBoard(log_dir=logdir, 
                                    histogram_freq=0, 
                                    batch_size = batch_size,
                                    write_graph=True, 
                                    write_grads=False)

        X_train = np.expand_dims(X_train, axis=2)
        X_valid = np.expand_dims(X_valid, axis=2)

        # Change the labels from categorical to one-hot encoding
        y_train_onehot = np.asarray(pd.get_dummies(y_train), dtype = np.int8)
        y_valid_onehot = np.asarray(pd.get_dummies(y_valid), dtype = np.int8)
        
        model_train =  self.combined.fit(X_train, y_train_onehot,
                                              batch_size=batch_size,epochs=epochs,verbose=0,
                                              validation_data=(X_valid, y_valid_onehot))
        
        return model_train

# Training 1D to 2D model

In [ ]:
def train_1d_to_2d():
  global OtT1
  global OtT2
  global OtT3
  OtT1 = OnetoTwoDimensionNetwork(rows=SX_train.shape[2], cols=1, channels=1, classes=n_driver)
  OtT2 = OnetoTwoDimensionNetwork(rows=SX_train.shape[2], cols=1, channels=1, classes=n_driver)
  OtT3 = OnetoTwoDimensionNetwork(rows=SX_train.shape[2], cols=1, channels=1, classes=n_driver)
  
  OtT1.train(epochs=20, X_train=SX_train[:,0,:], y_train=y_train, X_valid=SX_test[:,0,:], y_valid=y_test, batch_size=32)
  OtT2.train(epochs=20, X_train=SX_train[:,1,:], y_train=y_train, X_valid=SX_test[:,1,:], y_valid=y_test, batch_size=32)
  OtT3.train(epochs=20, X_train=SX_train[:,2,:], y_train=y_train, X_valid=SX_test[:,2,:], y_valid=y_test, batch_size=32)

In [ ]:
def evaluate_1d_to_2d():
  y_pred_probabilities1 = OtT1.combined.predict(np.expand_dims(SX_test[:,0,:], axis=2))
  y_pred_probabilities2 = OtT2.combined.predict(np.expand_dims(SX_test[:,1,:], axis=2))
  y_pred_probabilities3 = OtT3.combined.predict(np.expand_dims(SX_test[:,2,:], axis=2))
  
  y_pred_probabilities = np.add(y_pred_probabilities1, y_pred_probabilities2, y_pred_probabilities3)
  
  # decision_nb = 15
  # decision_time = (decision_nb-1)*((window_size-overlapping)/sample_rate)+window_size/sample_rate
  decision_time = 60 * 9
  decision_nb = int((decision_time - (window_size / sample_rate)) / ((window_size - overlapping) / sample_rate)) + 1
  print("decision time", str(timedelta(seconds=decision_time)))

  df = []
  y_real = []

  y_pred = np.asarray(pd.get_dummies(np.argmax(y_pred_probabilities, axis=1)), dtype = np.int8)
  for class_i in np.unique(y_test):
    subset = y_pred[np.where(y_test == class_i)]
    for i in range(0,subset.shape[0],int(decision_nb*0.75)):
      if i+decision_nb < subset.shape[0]:
        row = np.zeros(y_pred.shape[1])
        for j in range(decision_nb):
          row += subset[i+j]
        df.append(row)
        y_real.append(class_i)
  y_pred_labels = np.argmax(df, axis=1)
  accuracy = accuracy_score(y_real, y_pred_labels)
  precision = precision_score(y_real, y_pred_labels, average='macro')
  recall = recall_score(y_real, y_pred_labels, average='macro')
  f1 = f1_score(y_real, y_pred_labels, average='macro')
  print('Accuracy:{:.4f} Recall:{:.4f} Precision:{:.4f} F1:{:.4f}'.format(accuracy, recall, precision, f1))

In [ ]:
def generate_feature_maps():
  global feature_maps1
  global feature_maps2
  global feature_maps3
  
  feature_maps1 = OtT1.generator.predict(np.expand_dims(SX_train[:,0,:], axis=2))
  feature_maps2 = OtT2.generator.predict(np.expand_dims(SX_train[:,1,:], axis=2))
  feature_maps3 = OtT3.generator.predict(np.expand_dims(SX_train[:,2,:], axis=2))
  
  global X_train_fm
  X_train_fm = []
  for i in range(SX_train.shape[0]):
    row = []
    for j in range(SX_train.shape[2]):
      col = []
      for k in range(SX_train.shape[2]):
        cell = []
        cell.append(feature_maps1[i][j][k])
        cell.append(feature_maps1[i][j][k])
        cell.append(feature_maps1[i][j][k])
        col.append(cell)
      row.append(col)
    X_train_fm.append(row)
  X_train_fm = np.array(X_train_fm)

  feature_maps1 = OtT1.generator.predict(np.expand_dims(SX_test[:,0,:], axis=2))
  feature_maps2 = OtT2.generator.predict(np.expand_dims(SX_test[:,1,:], axis=2))
  feature_maps3 = OtT3.generator.predict(np.expand_dims(SX_test[:,2,:], axis=2))
  
  global X_test_fm
  X_test_fm = []
  for i in range(SX_test.shape[0]):
    row = []
    for j in range(SX_test.shape[2]):
      col = []
      for k in range(SX_test.shape[2]):
        cell = []
        cell.append(feature_maps1[i][j][k])
        cell.append(feature_maps1[i][j][k])
        cell.append(feature_maps1[i][j][k])
        col.append(cell)
      row.append(col)
    X_test_fm.append(row)
  X_test_fm = np.array(X_test_fm)

# DI Model Training

In [ ]:
def train_model():
  base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))
  for layer in base_model.layers:
    layer.trainable = False
  
  x = base_model.output
  x = Reshape(target_shape=((7, 7*2048)), name='reshape')(x)
  x = GRU(1024, return_sequences=True, kernel_initializer='he_normal', name='gru1')(x)
  x = GRU(10, return_sequences=True, go_backwards=True, kernel_initializer='he_normal', name='gru1_b')(x)
  x = Flatten()(x) 
  predictions = Dense(n_driver, activation='softmax')(x)
  model = Model(base_model.input, predictions)
  
  # this is the model we will train
  model = Model(inputs=base_model.input, outputs=predictions)
  
  # initiate Adam optimizer
  opt = optimizer = Adam(0.001, 0.9)
  
  # Let's train the model using Adam
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

  # model.summary()

  y_train_onehot = np.asarray(pd.get_dummies(y_train), dtype = np.int8)
  y_valid_onehot = np.asarray(pd.get_dummies(y_test), dtype = np.int8)
  history = model.fit(X_train_fm, y_train_onehot, batch_size=32, epochs=20, verbose=1,
                    validation_data=(X_test_fm, y_valid_onehot))
  
  y_pred_prob_resnet = model.predict(X_test_fm)
  
  df = []
  y_real = []
  
  decision_time = 60 * 9
  decision_nb = int((decision_time - (window_size / sample_rate)) / ((window_size - overlapping) / sample_rate)) + 1

  y_pred = np.asarray(pd.get_dummies(np.argmax(y_pred_prob_resnet, axis=1)), dtype = np.int8)
  for class_i in np.unique(y_test):
    subset = y_pred[np.where(y_test == class_i)]
    for i in range(0,subset.shape[0],int(decision_nb*0.75)):
      if i+decision_nb < subset.shape[0]:
        row = np.zeros(y_pred.shape[1])
        for j in range(decision_nb):
          row += subset[i+j]
        df.append(row)
        y_real.append(class_i)
  y_pred_labels = np.argmax(df, axis=1)
  accuracy = accuracy_score(y_real, y_pred_labels)
  precision = precision_score(y_real, y_pred_labels, average='macro')
  recall = recall_score(y_real, y_pred_labels, average='macro')
  f1 = f1_score(y_real, y_pred_labels, average='macro')
  return accuracy, precision, recall, f1

In [ ]:
def save_result(accuracy_list, recall_list, precision_list, f1_list, running_time):
  saving_path = '/content/drive/MyDrive/DriverIdentification/Log/Sánchez/'
  data = {'window size':window_size / (60*sample_rate),'overlap':overlapping / window_size, 
          'algoritm':'CNN-GRU', 'dataset':'eftekhari', 'drivers':n_driver, 'features':features}
  (accuracy_mean,accuracy_std) = (np.average(accuracy_list),np.std(accuracy_list))
  (recall_mean,recall_std) = (np.average(recall_list),np.std(recall_list))
  (precision_mean,precision_std) = (np.average(precision_list),np.std(precision_list))
  (f1_mean,f1_std) = (np.average(f1_list),np.std(f1_list))
  result = {
      'accuracy_mean':accuracy_mean,'accuracy_std':accuracy_std,
      'recall_mean':recall_mean,'recall_std':recall_std,
      'precision_mean':precision_mean,'precision_mean':precision_mean,
      'f1_mean':f1_mean,'f1_std':f1_std,
  }
  print('Mean Accuracy:{:.4f}({:.4f}) Mean Recall:{:.4f}({:.4f}) Mean Precision:{:.4f}({:.4f}) Mean F1:{:.4f}({:.4f})'.format(
      accuracy_mean,accuracy_std, recall_mean,recall_std, precision_mean,precision_std, f1_mean,f1_std))
  utils.save_result(saving_path=saving_path, result=result, data=data, running_time=running_time)

# RUN

In [ ]:
segments = np.arange(15)
# np.random.shuffle(segments)
segment_length = 2 * 60 * 5
data_segments = []
for i in range(15):
    start = segments[i] * segment_length
    end = segments[i] * segment_length + segment_length - 1
    data_segments.append((start, end))

for i in range(10):
  print('train proportions:', data_segments[0:i] + data_segments[i+6:15])
  print('test  proportions:', data_segments[i:i+6])
  print("==========================================")

train proportions: [(3600, 4199), (4200, 4799), (4800, 5399), (5400, 5999), (6000, 6599), (6600, 7199), (7200, 7799), (7800, 8399), (8400, 8999)]
test  proportions: [(0, 599), (600, 1199), (1200, 1799), (1800, 2399), (2400, 2999), (3000, 3599)]
train proportions: [(0, 599), (4200, 4799), (4800, 5399), (5400, 5999), (6000, 6599), (6600, 7199), (7200, 7799), (7800, 8399), (8400, 8999)]
test  proportions: [(600, 1199), (1200, 1799), (1800, 2399), (2400, 2999), (3000, 3599), (3600, 4199)]
train proportions: [(0, 599), (600, 1199), (4800, 5399), (5400, 5999), (6000, 6599), (6600, 7199), (7200, 7799), (7800, 8399), (8400, 8999)]
test  proportions: [(1200, 1799), (1800, 2399), (2400, 2999), (3000, 3599), (3600, 4199), (4200, 4799)]
train proportions: [(0, 599), (600, 1199), (1200, 1799), (5400, 5999), (6000, 6599), (6600, 7199), (7200, 7799), (7800, 8399), (8400, 8999)]
test  proportions: [(1800, 2399), (2400, 2999), (3000, 3599), (3600, 4199), (4200, 4799), (4800, 5399)]
train proportions: [

In [ ]:
for n_driver in range(4,5):
  accuracy_list = []
  recall_list = []
  precision_list = []
  f1_list = []
  start = datetime.now()
  for i in range(1):
    train_segments = data_segments[0:i] + data_segments[i+6:15]
    test_segments = data_segments[i:i+6]
    initialization(n_driver, train_segments, test_segments)
    read_data()
    replace_ids()
    data_standardization()
    feature_extraction()
    train_1d_to_2d()
    evaluate_1d_to_2d()
    generate_feature_maps()
    accuracy, precision, recall, f1 = train_model()
    accuracy_list.append(accuracy)
    precision_list.append(precision)
    recall_list.append(recall)
    f1_list.append(f1)
    print('Accuracy:{:.4f} Precision:{:.4f} Recall:{:.4f} F1:{:.4f}'.format(accuracy, precision, recall, f1))
  end = datetime.now()
  running_time = end - start
  save_result(accuracy_list, recall_list, precision_list, f1_list, running_time)

driver_id:          original_size:      stay_size:          driving_size:       cleaned_driving_size: 
207                 12:13:19            9:10:35             3:02:44             3:02:44             
208                 8:00:58             5:54:09             2:06:49             2:06:49             
209                 6:34:13             4:20:44             2:13:29             2:13:29             
210                 6:05:37             3:14:38             2:50:59             2:50:59             
segmenting data with 21564 points
making 124 segments
segmenting data with 14376 points
making 84 segments
decision time 0:09:00
Accuracy:0.7500 Recall:0.7500 Precision:0.5929 F1:0.6540
Epoch 1/20
4/4 [==============================] - 44s 10s/step - loss: 0.6190 - accuracy: 0.2327 - val_loss: 0.5527 - val_accuracy: 0.3333
Epoch 2/20
4/4 [==============================] - 35s 10s/step - loss: 0.5206 - accuracy: 0.5394 - val_loss: 0.5509 - val_accuracy: 0.3333
Epoch 3/20
4/4 [=============